In [8]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from glob import glob
from sklearn.metrics import accuracy_score

# Parameters
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32
EPOCHS = 50

# Function to create synthetic medical images and masks
def create_synthetic_data(image_dir, mask_dir, num_images=100):
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(mask_dir, exist_ok=True)

    for i in range(num_images):
        # Create a synthetic grayscale image
        image = np.random.randint(0, 256, (IMG_HEIGHT, IMG_WIDTH), dtype=np.uint8)
        cv2.imwrite(os.path.join(image_dir, f'image_{i}.png'), image)

        # Create a corresponding binary mask (random for demonstration)
        mask = (image > 128).astype(np.uint8) * 255  # Simple thresholding for mask
        cv2.imwrite(os.path.join(mask_dir, f'mask_{i}.png'), mask)

# Create synthetic data
image_dir = 'synthetic_images'
mask_dir = 'synthetic_masks'
create_synthetic_data(image_dir, mask_dir)

# Define U-Net model
def unet_model(input_size=(IMG_HEIGHT, IMG_WIDTH, 1)):
    inputs = layers.Input(input_size)

    # Contracting path
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c5)

    # Expansive path
    u6 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1])
    c9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model

# Load data function
def load_data(image_dir, mask_dir):
    images = []
    masks = []
    image_files = glob(os.path.join(image_dir, '*.png'))  # Adjust the extension as needed

    print(f'Found {len(image_files)} images in {image_dir}')

    for img_path in image_files:
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH), color_mode='grayscale')
        img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
        images.append(img)

        mask_path = os.path.join(mask_dir, os.path.basename(img_path).replace('image', 'mask'))  # Assuming masks have the same name
        if os.path.exists(mask_path):
            mask = tf.keras.preprocessing.image.load_img(mask_path, target_size=(IMG_HEIGHT, IMG_WIDTH), color_mode='grayscale')
            mask = tf.keras.preprocessing.image.img_to_array(mask) / 255.0
            masks.append(mask)
        else:
            print(f'Mask not found for {mask_path}')

    return np.array(images), np.array(masks)

# Load your data
images, masks = load_data(image_dir, mask_dir)

# Proceed only if images and masks are loaded
if images.size > 0 and masks.size > 0:
    # Split the data
    X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.2, random_state=42)

    # Data augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Compile the model
    model = unet_model()
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model with data augmentation
    model.fit(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
              validation_data=(X_val, y_val),
              epochs=EPOCHS,
              steps_per_epoch=len(X_train) // BATCH_SIZE)

    # Evaluate the model
    val_predictions = model.predict(X_val)
    val_predictions = (val_predictions > 0.5).astype(np.uint8)

    # Calculate accuracy (or any other metric)
    accuracy = accuracy_score(y_val.flatten(), val_predictions.flatten())
    print(f'Validation accuracy: {accuracy:.2f}')
else:
    print("No data to process. Please check your directories.")

Found 100 images in synthetic_images
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2/2 ━━━━━━━━━━━━━━━━━━━━ 49s 21s/step - accuracy: 0.4990 - loss: 0.6933 - val_accuracy: 0.5041 - val_loss: 0.6934
Epoch 2/50
1/2 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.5039 - loss: 0.6932

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.5039 - loss: 0.6932 - val_accuracy: 0.5048 - val_loss: 0.6930
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 60s 11s/step - accuracy: 0.5039 - loss: 0.6931 - val_accuracy: 0.5491 - val_loss: 0.6925
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 18s 2s/step - accuracy: 0.5019 - loss: 0.6931 - val_accuracy: 0.5812 - val_loss: 0.6923
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 36s 20s/step - accuracy: 0.5006 - loss: 0.6931 - val_accuracy: 0.5720 - val_loss: 0.6920
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - accuracy: 0.5022 - loss: 0.6931 - val_accuracy: 0.5319 - val_loss: 0.6918
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 72s 19s/step - accuracy: 0.5036 - loss: 0.6931 - val_accuracy: 0.5065 - val_loss: 0.6917
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 5s/step - accuracy: 0.5026 - loss: 0.6932 - val_accuracy: 0.5076 - val_loss: 0.6918
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 34s 18s/step - accuracy: 0.5039 - loss: 0.6931 - val_accuracy: 0.5173 - val_loss: 0.6920
Epoch 10/50
2/2